# Notebook de construction du dataset

Dans ce notebook, nous allons construire le dataset qui nous servira a entrainer notre modèle de prédiction d'affluence à la cantine ainsi que notre modèle de prédiction du gaspillage alimentaire.

In [1]:
import pandas as pd
import numpy as np

# Read in the data

df = pd.read_csv('data/original.csv')

df_matching = pd.read_csv('data/food_matching.csv')

# Remove bio column (all empty)
df = df.drop(['bio'], axis=1)

# Remove rows with missing values
df = df.dropna()

# Check if there is still missing values
df.isnull().sum()

# Split the menu column into multiple columns
df_menu = df['menu'].str.split(',', expand=True)

# Remove all [ ] and ' ' from all columns
df_menu = df_menu.replace(to_replace=r'\[|\]|\'|\"', value='', regex=True)

# Append df_menu to df
df = df.join(df_menu)

# Remove menu column
#df = df.drop(['menu'], axis=1)

# Rename columns
for i in range(0, df_menu.shape[1]):
    df = df.rename(columns={i: 'menu' + str(i)})
    df_menu = df_menu.rename(columns={i: 'menu' + str(i)})

# Fill NaN values with ""
df = df.fillna("")

df.head()

,date,prevision,reel,effectif,taux_grevistes,nos,ind,greves_manquantes,menu,ferie,...,menu4,menu5,menu6,menu7,menu8,menu9,menu10,menu11,menu12,menu13
0,2011-01-03,12289.0,11129.0,17373.0,1.1,1.0,0.0,0.0,"['Pamplemousse', 'Hachis Parmentier', 'Salade ...",0.0,...,,,,,,,,,,
1,2011-01-04,12558.0,11808.0,17373.0,0.0,0.0,0.0,0.0,"['Crêpe au fromage', 'emincé de volaille à la ...",0.0,...,ananas,,,,,,,,,
2,2011-01-06,12753.0,11741.0,17373.0,0.0,0.0,0.0,0.0,"['thon', 'Mâche au thon', 'Croziflette', 'des ...",0.0,...,des de volaille,Yaourt brasse bio,,,,,,,,
3,2011-01-07,12812.0,12322.0,17373.0,0.0,0.0,0.0,0.0,"['Choux blancs mimolette', 'milanette en des',...",0.0,...,Galette des rois,Lait fermier,,,,,,,,
4,2011-01-10,12309.0,11302.0,17373.0,0.0,0.0,0.0,0.0,"['Endives / demi-oeuf', 'paella de la mer', 'F...",0.0,...,,,,,,,,,,


In [2]:
# remove every start and end space in df_menu
for i in range(0, df_menu.shape[1]):
    df_menu['menu' + str(i)] = df_menu['menu' + str(i)].str.strip()

In [3]:
# Match the menu items with the Meal Type and Food type in the matching table

# Create a new column for each menu item
df.to_csv('data/original_processed.csv', index=False)

In [4]:
# Replace every occurences of dessert with Dessert

df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='dessert', value='Dessert', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='légume', value='Accompagnement', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='fromage', value='Fromage', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='Boisson', value='Dessert', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='Condiment', value='Accompagnement', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='Sauce', value='Accompagnement', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='Fromsert', value='Dessert', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='Garniture', value='Accompagnement', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='Entée', value='Entrée', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='charcuterie', value='Entrée', regex=True)
df_matching['Meal Type'] = df_matching['Meal Type'].replace(
    to_replace='fête', value=np.nan, regex=True)


df_matching['Meal Type'].unique()


array(['Dessert', 'Entrée', 'Plat', 'Accompagnement', 'Fromage', nan],
      dtype=object)

In [5]:
df_matching.dropna(inplace=True)
df_matching['Meal Type'].unique()

array(['Dessert', 'Entrée', 'Plat', 'Accompagnement', 'Fromage'],
      dtype=object)

In [6]:
# Load the Menus and Matching DataFrames
menus_df = pd.read_csv('data/original_processed.csv')
matching_df = df_matching

# Create a list of 'menuX' column names
menu_columns = [f'menu{i}' for i in range(14)]

# Initialize empty columns for each Meal Type and Food Type
meal_types = ['Entrée', 'Plat', 'Accompagnement', 'Fromage', 'Dessert']
food_types = ['Entrée Food Type', 'Plat Food Type', 'Accompagnement Food Type', 'Fromage Food Type', 'Dessert Food Type']

for column in meal_types + food_types:
    menus_df[column] = None

# Create flag columns for each Meal Type
for meal_type in meal_types:
    menus_df[f'{meal_type}_Bio'] = 0

# Iterate through the 'menuX' columns and populate the corresponding meal type, food type, and flag columns
for col_name in menu_columns:
    menus_df[col_name] = menus_df[col_name].apply(lambda x: x.strip() if isinstance(x, str) else x)
    for i in range(len(menus_df)):
        menu_item = menus_df.at[i, col_name]
        match = matching_df[matching_df['Menu item'] == menu_item]
        if not match.empty:
            meal_type = match['Meal Type'].values[0]
            food_type = match['Food Type'].values[0]
            menu_item_with_food = f"{menu_item}"
            if menus_df.at[i, meal_type] is None:
                menus_df.at[i, meal_type] = [menu_item_with_food]
            else:
                menus_df.at[i, meal_type].append(menu_item_with_food)
            if menus_df.at[i, food_types[meal_types.index(meal_type)]] is None:
                menus_df.at[i, food_types[meal_types.index(meal_type)]] = [food_type]
            else:
                menus_df.at[i, food_types[meal_types.index(meal_type)]].append(food_type)
            if 'bio' in menu_item_with_food.lower():
                menus_df.at[i, f'{meal_type}_Bio'] = 1

# Drop the 'menuX' columns
menus_df = menus_df.drop(menu_columns, axis=1)

In [7]:
# Print the first 5 rows of the result
menus_df.head()

,date,prevision,reel,effectif,taux_grevistes,nos,ind,greves_manquantes,menu,ferie,...,Entrée Food Type,Plat Food Type,Accompagnement Food Type,Fromage Food Type,Dessert Food Type,Entrée_Bio,Plat_Bio,Accompagnement_Bio,Fromage_Bio,Dessert_Bio
0,2011-01-03,12289.0,11129.0,17373.0,1.1,1.0,0.0,0.0,"['Pamplemousse', 'Hachis Parmentier', 'Salade ...",0.0,...,"[agrumes, legumes autres]",[pomme de terre],None,None,[Lait],0,0,0,0,0
1,2011-01-04,12558.0,11808.0,17373.0,0.0,0.0,0.0,0.0,"['Crêpe au fromage', 'emincé de volaille à la ...",0.0,...,None,"[Blé Seigle, Volaille]",[Legumineuse autre],[Fromage],[fruits autre],0,0,0,0,0
2,2011-01-06,12753.0,11741.0,17373.0,0.0,0.0,0.0,0.0,"['thon', 'Mâche au thon', 'Croziflette', 'des ...",0.0,...,"[legumes autres, Volaille, Volaille]","[Poissons, pomme de terre]",None,[Lait],None,0,0,0,1,0
3,2011-01-07,12812.0,12322.0,17373.0,0.0,0.0,0.0,0.0,"['Choux blancs mimolette', 'milanette en des',...",0.0,...,"[type chou, Blé Seigle]","[Poissons, Blé Seigle]",[legumes racines],[Lait],None,0,0,0,0,0
4,2011-01-10,12309.0,11302.0,17373.0,0.0,0.0,0.0,0.0,"['Endives / demi-oeuf', 'paella de la mer', 'F...",0.0,...,[oeufs],[Riz],None,[Lait],[Bais et Raisins],0,0,0,0,0


In [8]:
# Check if there are '[]' values in the 'menu' columns and delete them
menus_df = menus_df[menus_df['menu'] != '[]']


In [9]:
# Export the result to a CSV file
menus_df.to_csv('data/final_dataset.csv', index=False)